In [5]:
import win32com.client as win32
import openpyxl 
import pandas as pd 
import ctypes 
import win32com 
import time 
import shutil 
import warnings 
import os 
import numpy as np 
import datetime 
import xlwings as xw
import calendar
from pathlib import Path
from openpyxl.styles import Font, NamedStyle, Alignment

dd = datetime.date.today().strftime("%d.%m.%Y") 

df=pd.read_excel(f'file_cov/ВВБ_AntiCOV_v2_{dd}.xlsx', sheet_name='Лист1', 
               usecols=["Просрочена ревакцинация", "Прогноз ревакцинации на неделю", "Табельный номер руководителя", "ФИО"])

df["ФИО"]=np.where((df["Просрочена ревакцинация"]==1)|(df["Прогноз ревакцинации на неделю"]==1), df["ФИО"], np.nan) 
df=df[df["ФИО"].notnull()]
df=df.sort_values(by=["Табельный номер руководителя"])


#выгрузка данных в шаблон wsa из Коллективного иммунитета df["ФИО"]

del df["Просрочена ревакцинация"] 
del df["Прогноз ревакцинации на неделю"]

df["Внутренний e-mail руков"]="" 
df["Внешний e-mail руков"]=""
df["Нумерация"]=""
df["Сцепка1"]=""
df["Сцепка2"]=""
df["Выбор"]=""
df["Сообщение"]=""

df.to_excel(f'ovbRBOSS/Обработчик к ревакцинации для руковод.xlsx', index=False)

print("готово1")


spr = str(Path.cwd() / f'spr/Справочник.xlsx') #№1 таблица Справочник
name_s = os.path.basename(spr)
shabl = str(Path.cwd() / f'ovbRBOSS/Обработчик к ревакцинации для руковод.xlsx') 
file_cov = str(Path.cwd() /f'ovbRBOSS/Обработчик к ревакцинации для руковод2.xlsx')  #сформированный файл

excel = win32.Dispatch("Excel.Application") #интеграция
ex = True  # Поставить False для итогового варианта
excel.Visible = ex
excel.DisplayAlerts = ex
excel.ScreenUpdating = ex

print('загрузка данных')

#чтение таблицы 1 
df2 = pd.read_excel(spr, sheet_name='Справочник')

df2.fillna("-", inplace=True) 
df2['Табельный номер руководителя'] = df2['Табельный номер руководителя'].astype(int)
df2 = df2.rename(columns={"Табельный номер руководителя": "ТН"})

print('DF1 chisl - Done')
lst_t1 = df2.ТН.tolist()
print("список ТН - Done") 
df2 = df2.set_index('ТН') 

print('download shablon')
start_time_2 = time.time()  # время запуска
wb = openpyxl.load_workbook(shabl)

ws = wb.active
n = 0
sheets = wb.sheetnames
wsa = wb[sheets[n]]
# print(wsa)
rows = wsa.max_row 
cols = wsa.max_column 
# wsa.sheet_properties.tabColor = '66FF00'
print('Запускается перебор строк')

#выгрузка данных в шаблон wsa 
for i in range(2, rows+1):
    if wsa.cell(row=i, column=2).value in lst_t1:
        wsa.cell(row=i, column=3).value = df2.loc[wsa.cell(row=i, column=2).value, "Внутренний e-mail руков"]
        wsa.cell(row=i, column=4).value = df2.loc[wsa.cell(row=i, column=2).value, "Внешний e-mail руков"]
    else: wsa.cell(row=i, column=3).value=0
        
    if wsa.cell(row=i, column=2).value==wsa.cell(row=i-1, column=2).value:
        wsa.cell(row=i, column=5).value=wsa.cell(row=i-1, column=5).value+1
    else: wsa.cell(row=i, column=5).value=1
    
   # wsa.cell(row=i, column=4).value=str(wsa.cell(row=i, column=4).value)
   # wsa.cell(row=i, column=3).value=str(wsa.cell(row=i, column=3).value)
for i in range(2, rows+1):    
    if wsa.cell(row=i, column=4).value!='' and wsa.cell(row=i, column=5).value!='':
        wsa.cell(row=i, column=5).value=str(wsa.cell(row=i, column=5).value)
        wsa.cell(row=i, column=1).value=str(wsa.cell(row=i, column=1).value)
        wsa.cell(row=i, column=6).value="<strong>"+(wsa.cell(row=i, column=5).value)+"."+" "+(wsa.cell(row=i, column=1).value)+"</strong>"+"<br>"+"<br>"

    if wsa.cell(row=i, column=2).value==wsa.cell(row=i-1, column=2).value and wsa.cell(row=i, column=7).value!="":
        wsa.cell(row=i, column=6).value=str(wsa.cell(row=i, column=6).value)
        wsa.cell(row=i-1, column=7).value=str(wsa.cell(row=i-1, column=7).value)
        wsa.cell(row=i, column=7).value=wsa.cell(row=i-1, column=7).value+wsa.cell(row=i, column=6).value
    else: wsa.cell(row=i, column=7).value=wsa.cell(row=i, column=6).value
        
for i in range(2, rows+1): 
    arg1=str(wsa.cell(row=i+1, column=7).value)
    arg2=str(wsa.cell(row=i, column=7).value)
    if len(arg2)<len(arg1):
        wsa.cell(row=i, column=8).value=0
    else: wsa.cell(row=i, column=8).value=1

for i in range(2, rows+1): 
    if wsa.cell(row=i, column=8).value==1:
        wsa.cell(row=i, column=9).value="Сообщаем о необходимости прохождения ревакцинации и передачи данных в СБЕРДРУГ для сотрудников вашего подразделения:"+"<br>"+"<br>"+(wsa.cell(row=i, column=7).value)
    else: wsa.cell(row=i, column=9).value=""
print('переменные')


wb.save(file_cov)
ext = True
excel.Visible = ext
excel.DisplayAlerts = ext
excel.ScreenUpdating = ext

print(f'save - {file_cov}')


df=pd.read_excel(f'ovbRBOSS/Обработчик к ревакцинации для руковод2.xlsx', sheet_name='Sheet1')

df["ФИО"]=np.where((df["Выбор"]!=0), df["ФИО"], np.nan) 
df=df[df["ФИО"].notnull()]

df["ФИО"]=np.where((df["Табельный номер руководителя"]!=0), df["ФИО"], np.nan) 
df=df[df["ФИО"].notnull()]

df["ФИО"]=np.where((df["Внутренний e-mail руков"]!=0), df["ФИО"], np.nan) 
df=df[df["ФИО"].notnull()]

#выгрузка данных в шаблон wsa из Коллективного иммунитета df["ФИО"]

del df["ФИО"] 
del df["Табельный номер руководителя"]
del df["Нумерация"]
del df["Сцепка1"]
del df["Сцепка2"]
del df["Выбор"]

df.to_excel(f'ovbRBOSS/Обработчик к ревакцинации для руковод3.xlsx', index=False)

print("готово3")

готово1
загрузка данных
DF1 chisl - Done
список ТН - Done
download shablon
Запускается перебор строк
переменные
save - C:\Users\19356099\Documents\В РАЗРАБОТКЕ\Обработчик ревакцинация для руководителей\ovbRBOSS\Обработчик к ревакцинации для руковод2.xlsx
готово3
